In [73]:
import pandas as pd
import json

In [74]:
#train_data=pd.read_json('/Users/suprajagandhi/Downloads/deliverable-dataset/training-set-100000.json')

In [75]:
# Read the json file and convert to json dict
with open("/Users/suprajagandhi/Downloads/deliverable-dataset/training-set-100000.json", "r") as file:
    data = json.load(file)

In [76]:
#Converting json dictionary to dataframe
tr_data = pd.DataFrame(data["articles"])
tr_data.head()

,abstractText,journal,meshMajor,pmid,title,year
0,To clarify the role of endothelial cells in th...,"Journal of neurology, neurosurgery, and psychi...","[Aged, Autoantibodies, Biopsy, Cell Adhesion M...",1372348,Endothelial cell activation in vasculitis of p...,1992
1,AIMS: In light of recently found contribution ...,European journal of surgical oncology : the jo...,"[Adult, Aged, Aged, 80 and over, Carcinoma, Sq...",17174061,The interleukin-8 (-251A/T) polymorphism is as...,2007
2,The growth potential of normal and transformed...,Cancer letters,"[Animals, Cell Line, Cell Transformation, Neop...",544009,Identification of transformed rat liver epithe...,1979
3,"In the absence of autopsy studies, the etiolog...",Archives des maladies du coeur et des vaisseaux,"[Arrhythmias, Cardiac, Death, Sudden, Cardiac,...",8267507,[Primary ventricular fibrillation].,1993
4,A method has been developed which is able to q...,Journal of chromatography. A,"[Amino Acid Sequence, Electrophoresis, Capilla...",9764495,Quantifying biosynthetic human growth hormone ...,1998


In [77]:
print("Rows : " + str(tr_data.shape[0]))
print("Columns : " + str(tr_data.shape[1]))
print(tr_data.columns)

Rows : 100000
Columns : 6
Index(['abstractText', 'journal', 'meshMajor', 'pmid', 'title', 'year'], dtype='object')


In [78]:
#Considering only 1000 rows to improve performance
tr_data = tr_data[:1000]

In [79]:
input_data = tr_data['title'] + ' ' + tr_data['abstractText']
output_data = tr_data['meshMajor']
print(output_data)


0      [Aged, Autoantibodies, Biopsy, Cell Adhesion M...
1      [Adult, Aged, Aged, 80 and over, Carcinoma, Sq...
2      [Animals, Cell Line, Cell Transformation, Neop...
3      [Arrhythmias, Cardiac, Death, Sudden, Cardiac,...
4      [Amino Acid Sequence, Electrophoresis, Capilla...
                             ...                        
995    [ADP-Ribosylation Factors, Adaptor Proteins, V...
996    [Cell Membrane, Chromatography, Chromatography...
997    [Calmodulin, Chromatography, Affinity, Fluores...
998    [Art Therapy, Child, Fantasy, Female, Foster H...
999    [Aged, Anticoagulants, Antithrombins, Brain Is...
Name: meshMajor, Length: 1000, dtype: object


In [80]:
print(len(output_data[0]))
print(len(output_data[1]))
#Mesh major column is a array of varying lengths

18
19


In [81]:
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix
binarizer = MultiLabelBinarizer(sparse_output=True)

In [82]:
output_binary = binarizer.fit_transform(output_data)
outputbd = csr_matrix(output_binary).todense()

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),  
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

In [106]:
pipe.fit(input_data, outputbd)

/Users/suprajagandhi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [85]:
# Read the test data json file and convert to json dict
with open("/Users/suprajagandhi/Downloads/deliverable-dataset/test-set-20000-rev2.json", "r") as file:
    test_data = json.load(file)

In [86]:
#Converting json dictionary to dataframe
te_data = pd.DataFrame(test_data["documents"])
te_data.head()

,pmid,title,abstractText,meshMajor
0,6825728,"Superoxide dismutase, catalase and glutathione...","The activities of the protective enzymes, supe...","[Aged, Animals, Catalase, Cataract, Cattle, Gl..."
1,1275705,Division of the pyriformis muscle for the trea...,Division of the pyriformis muscle at its tendi...,"[Adult, Aged, Humans, Laminectomy, Lumbar Vert..."
2,19268353,Maternal and child health in the occupied Pale...,The Countdown to 2015 intervention coverage in...,"[Adolescent, Adult, Birth Rate, Child Health S..."
3,8706361,The effect of ultra-low sodium dialysate in CA...,"In peritoneal dialysis, sodium (Na) passes thr...","[Adult, Biological Transport, Chronic Disease,..."
4,2041353,[Surgical treatment of extensive hemangiomas o...,The results of treatment of 7 patients with ex...,"[Adult, Age Factors, Aged, Female, Hemangioma,..."


In [87]:
te_input = te_data['title'] + ' ' + te_data['abstractText']
te_output = te_data['meshMajor']
print(te_output)


0        [Aged, Animals, Catalase, Cataract, Cattle, Gl...
1        [Adult, Aged, Humans, Laminectomy, Lumbar Vert...
2        [Adolescent, Adult, Birth Rate, Child Health S...
3        [Adult, Biological Transport, Chronic Disease,...
4        [Adult, Age Factors, Aged, Female, Hemangioma,...
                               ...                        
19995    [Animals, Antigens, Helminth, Carbohydrate Con...
19996    [Activities of Daily Living, Adolescent, Adult...
19997    [Animals, Body Weight, Copper, Diet, Female, I...
19998    [Animals, Antibodies, Viral, Body Weight, Cerc...
19999    [Anthrax, Bacillus anthracis, Clostridium, Con...
Name: meshMajor, Length: 20000, dtype: object


In [92]:
te_output_binary = binarizer.fit_transform(te_output)
te_outputbd = csr_matrix(te_output_binary).todense()

In [ ]:
predicted_labels = pipe.predict(te_input)

In [ ]:
#pred_labels_inv = binarizer.inverse_transform(predicted_labels)

In [102]:
print(predicted_labels.shape)
print(te_outputbd.shape)

(20000, 4550)
(20000, 16247)


In [110]:
from sklearn.metrics import accuracy_score
#Accuracy
acc = accuracy_score(te_outputbd, predicted_labels)
print("Accuracy is " +str(acc))

Accuracy is 0.0


In [112]:
from sklearn.metrics import precision_score
mp = precision_score(te_output, predicted_labels, average='micro')
print("Micro precision is" +str(mp))

Micro precision is0.7564935064935064


In [113]:
from sklearn.metrics import recall_score
mr = recall_score(te_output, predicted_labels, average='micro')
print("Micro Recall is" +str(mr))

Micro Recall is0.09048543689320389


In [114]:
from sklearn.metrics import f1_score
mf = f1_score(te_output, predicted_labels, average='micro')
print("F1 score is" +str(mf))

F1 score is0.16163718348942074


In [115]:
# Read the judge data json file and convert to json dict
with open("/Users/suprajagandhi/Downloads/deliverable-dataset/judge-set-10000-unannotated.json", "r") as file:
    judge_data = json.load(file)

In [116]:
#Converting json dictionary to dataframe
judge_data = pd.DataFrame(judge_data["documents"])
judge_data.head()

,pmid,title,abstractText
0,16854706,The use of seat belts in cars with smart seat ...,"UNLABELLED: Recently, smart seat belt reminder..."
1,12943287,Alignment in total knee arthroplasty following...,In total knee arthroplasty (TKA) following fai...
2,9808709,The expression of heme oxygenase-1 gene respon...,"Phorone, a glutathione (GSH) depletor, induces..."
3,3856646,Tympanic plate fracture from mandibular trauma.,A case in which trauma to the mandibular midli...
4,22033210,Escherichia coli lipopolysaccharides and Staph...,MicroRNAs (miRNAs) are a family of regulatory ...


In [117]:
judge_input = judge_data['title'] + ' ' + judge_data['abstractText']

In [119]:
pred_judge_labels = pipe.predict(judge_input)

In [ ]:
labels = binarizer.inverse_transform(pred_judge_labels)

In [ ]:
#output JSON
pmid = judge_data["pmid"].tolist()
output_JSON = {"documents": [{"pmid": pmid, "labels": list(labels)} for pmid, labels in zip(pmid, labels)}

In [ ]:
with open("judge_labels_predicted.json", "w") as json_file:
    json.dump(output_JSON, json_file)